## Loading a model from a check-point and defining the detection function

In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [2]:
config_path=r"C:\Tensor_Flow\Final_Thesis\drive\MyDrive\models\Near_Final_resized_800\my_ssd_mobnet\pipeline.config"
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(config_path)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt_path=r"C:\Tensor_Flow\Final_Thesis\drive\MyDrive\models\Near_Final_resized_800\my_ssd_mobnet"
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(ckpt_path, 'ckpt-41')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [4]:
#define the path in which your photos are located
rout_path=r"C:\Users\Admin\Desktop\Mostafa_F\drugs_camera"
#define the path in which your 'label_map.pbtxt' are located
label_map_path=os.path.join(rout_path, 'label_map.pbtxt')
category_index = label_map_util.create_category_index_from_labelmap(label_map_path)

## Using the detection function to produce bounding boxes in the pictures.

In [5]:
import xml.etree.ElementTree as ET
import copy
import math
import pandas as pd
import glob as glob
import cv2
import numpy as np

In [7]:
#Define the minimum probability of being an object to draw a box around it
Threshold=0.3
#Define the directory in which your pictures are located
IMAGE_PATH = os.path.join(rout_path, '*.jpg')
#Define the directory in which your example annotation "xml" file is located
path=r"C:\Users\Admin\Desktop\Mostafa_F\drugs_camera\example.xml"
for file in glob.glob(IMAGE_PATH):
    img = cv2.imread(file)
    image_np = np.array(img)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detection_scores_pd=pd.DataFrame(detections['detection_scores'],columns = ['score'])
    detection_boxes_pd=pd.DataFrame(detections['detection_boxes'],columns = ['xmin','ymin','xmax','ymax'])
    detection_classes_pd=pd.DataFrame(detections['detection_classes'],columns = ['class'])
    detection_scores_pd.rename({0: "Score"}, axis=1)
    detection_data=pd.concat([detection_boxes_pd,detection_scores_pd],axis=1)
    detection_data=pd.concat([detection_data,detection_classes_pd],axis=1)
    #The minimum score threshold is defined here
    detection_data_thresholded=detection_data[detection_data["score"]>Threshold]
    #import xml file
    tree = ET.parse(path)
    root = tree.getroot()
    tree_dupe=copy.deepcopy(tree)
    elems = tree.findall(".//filename")
    for elem in elems:
        elem.text = os.path.basename(IMAGE_PATH[0:-4])+'.jpg'
    elems = tree.findall(".//object")
    #elems.pop()
    for elem in elems:
        root.remove(elem)
        dupe = copy.deepcopy(elem)
    tree_dupe=copy.deepcopy(tree)
    root_dupe = tree_dupe.getroot()
    elems = tree_dupe.findall(".//object")
    for elem in elems:
        root_dupe.remove(elem)
    elems = tree_dupe.findall(".//height")
    for elem in elems:
        elem.text = str(img.shape[0])
    elems = tree_dupe.findall(".//width")
    for elem in elems:
        elem.text = str(img.shape[1])
    for ind in detection_data_thresholded.index:
        obj=copy.deepcopy(dupe)
        for val in obj.iter('name'):
            if detection_data_thresholded['class'][ind]==0.0:
                val.text="Filled"
            else:
                val.text="Empty" 
        for val in obj.iter('xmin'):
            val.text=str(math.floor(img.shape[1]*detection_data_thresholded['ymin'][ind]))
        for val in obj.iter('xmax'):
            val.text=str(math.floor(img.shape[1]*detection_data_thresholded['ymax'][ind]))
        for val in obj.iter('ymin'):
            val.text=str(math.floor(img.shape[0]*detection_data_thresholded['xmin'][ind]))
        for val in obj.iter('ymax'):
            val.text=str(math.floor(img.shape[0]*detection_data_thresholded['xmax'][ind]))
        root_dupe.append(obj)
    out_path=os.path.join(rout_path,os.path.basename(file[0:-4])+'.xml')
    tree_dupe.write(out_path)



## Install and open labelimg to correct for inaccuracies

In [1]:
import os

In [2]:
!pip install --upgrade pyqt5 lxml


     |████████████████████████████████| 8.3 MB 15.6 MB/s 
     |████████████████████████████████| 6.3 MB 27.6 MB/s 
     |████████████████████████████████| 317 kB 62.1 MB/s 
     |████████████████████████████████| 59.9 MB 1.3 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
LABELIMG_PATH = r"C:\Tensor Flow for object detection\Tensorflow\labelimg"

In [4]:
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference repository
    --reference-if-able <repo>
                          reference repository
    --dissociate          use --reference only 

In [5]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

make: *** No rule to make target 'qt5py3'.  Stop.


In [8]:
!cd {LABELIMG_PATH} && python labelImg.py

The system cannot find the path specified.
